In [ ]:
# ONS (Census 2011) statistics for Lewisham.
# Fetches Nomis data releases and prepares them for use in Lewisham ward profiles.
#
# This requires the OA->LSOA lookup table produced by "lbl_boundaries_oa.ipynb"
# as well as the OA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [1]:
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the Nomis source data.

In [2]:
# Extract the subset we need.
def filter_nomis_data(d, date_values, geography_values):
  return d[d.date.isin(date_values) & 
           d.geography.isin(geography_values)]

# Reformat Nomis data into a lookup table format:
# clear labelling of the index (geography) column, 
# remove superfluous columns.
def format_nomis_data(d, geo_colname='OA11CD'):
  return d.\
           rename(columns={'geography code': geo_colname}).\
           drop(columns=['date', 'geography'])

# Convenience function to call the above.
def nomis_to_oa(d, years, oa_list):
  return format_nomis_data(filter_nomis_data(d, years, oa_list))

In [3]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
oa_lookups_dir = f"{project_dir}/lookups/oa"
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
output_dir = f"{project_dir}/religion"

In [6]:
!mkdir -p '{output_dir}'

Downloads
--

In [7]:
# Religion (detailed), OAs in London 2011
# Source: 
# https://www.nomisweb.co.uk/census/2011/qs210ew
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_573_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_religion_oa11_london.csv"

--2021-12-09 11:54:15--  https://www.nomisweb.co.uk/api/v01/dataset/nm_573_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.3.145
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.3.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/religion/census11_religion_oa11_london.csv’

/content/gdrive/MyD     [                <=> ]   3.96M   309KB/s    in 13s     

2021-12-09 11:54:31 (307 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/religion/census11_religion_oa11_london.csv’ saved [4154976]



In [8]:
# reference regions
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_573_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_religion_references.csv"

--2021-12-09 11:54:35--  https://www.nomisweb.co.uk/api/v01/dataset/nm_573_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.3.145
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.3.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/religion/census11_religion_references.csv’

/content/gdrive/MyD     [ <=>                ]   4.02K  --.-KB/s    in 0.07s   

2021-12-09 11:54:35 (57.4 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/religion/census11_religion_references.csv’ saved [4114]



Lookups
--
Used to match up OAs with their LSOAs, and 2022 electoral wards

In [9]:
# All OAs and their LSOAs in Lewisham
oa_lsoa_join = pd.read_csv(f"{oa_lookups_dir}/lbl_oa11_lsoa11_msoa11_lad20_rgn20_202012.csv")
oa_lsoa_join = oa_lsoa_join[['OA11CD', 'LSOA11CD']]
oa_lsoa_join.head()

,OA11CD,LSOA11CD
0,E00016277,E01003220
1,E00016278,E01003220
2,E00016285,E01003220
3,E00016257,E01003221
4,E00016263,E01003221


In [10]:
# All OAs and their 2022 wards in Lewisham
oa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_oa11_wd22_proposed.csv")
oa_wd22_join = oa_wd22_join[['OA11CD', 'WD22CD_proposed']]
oa_wd22_join.head()

,OA11CD,WD22CD_proposed
0,E00016403,E05013721
1,E00016442,E05013721
2,E00016407,E05013721
3,E00016404,E05013721
4,E00016402,E05013721


Process & export
==
Lewisham
--

In [11]:
# For filtering of source data
lbl_oa_list = oa_lsoa_join.OA11CD.unique() # All OAs in Lewisham
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['religion']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_oa11_london.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')

  # Transform to OA index data
  lbl_oa = nomis_to_oa(d, years, lbl_oa_list)
  lbl_oa.to_csv(f"{output_dir}/lbl_{datname}_oa11.csv", index=False)

  # Aggregate to LSOA level
  lbl_lsoa = aggregate_oa_groups(lbl_oa, oa_lsoa_join, oa_col='OA11CD', group_col='LSOA11CD')
  lbl_lsoa.to_csv(f"{output_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(lbl_lsoa.head())

  # Aggregate to WD22 level
  lbl_wd22 = aggregate_oa_groups(lbl_oa, oa_wd22_join, oa_col='OA11CD', group_col='WD22CD_proposed')
  lbl_wd22.to_csv(f"{output_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(lbl_wd22.head())

'=== religion ==='

,All categories: Religion,Christian,Buddhist,Hindu,Jewish,Muslim (Islam),Sikh,Other religion: Total,Other religion: Animism,Other religion: Baha'i,Other religion: Believe in God,Other religion: Brahma Kumari,Other religion: Chinese Religion,Other religion: Church of All Religion,Other religion: Confucianist,Other religion: Deist,Other religion: Druid,Other religion: Druze,Other religion: Eckankar,Other religion: Heathen,Other religion: Jain,Other religion: Mixed Religion,Other religion: Mysticism,Other religion: Native American Church,Other religion: New Age,Other religion: Occult,Other religion: Own Belief System,Other religion: Pagan,Other religion: Pantheism,Other religion: Rastafarian,Other religion: Ravidassia,Other religion: Reconstructionist,Other religion: Satanism,Other religion: Scientology,Other religion: Shamanism,Other religion: Shintoism,Other religion: Spiritual,Other religion: Spiritualist,Other religion: Taoist,Other religion: Theism,Other religion: Thelemite,Other religion: Traditional African Religion,Other religion: Unification Church,Other religion: Universalist,Other religion: Vodun,Other religion: Wicca,Other religion: Witchcraft,Other religion: Zoroastrian,Other religion: Other religions,No religion: Total,No religion: No religion,No religion: Agnostic,No religion: Atheist,No religion: Free Thinker,No religion: Heavy Metal,No religion: Humanist,No religion: Jedi Knight,No religion: Realist,Religion not stated
LSOA11CD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E01003189,1670,931,3,51,2,127,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,383,375,1,0,0,0,0,7,0,169
E01003190,1878,1082,17,36,3,141,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,415,412,0,2,0,0,0,1,0,175
E01003191,1494,857,14,15,0,88,2,8,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,1,0,1,378,369,2,2,0,0,0,5,0,132
E01003192,1639,944,9,47,6,110,2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,367,363,1,0,0,0,0,3,0,144
E01003193,1568,1001,14,13,3,70,6,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,316,312,0,0,0,0,0,4,0,141


,All categories: Religion,Christian,Buddhist,Hindu,Jewish,Muslim (Islam),Sikh,Other religion: Total,Other religion: Animism,Other religion: Baha'i,Other religion: Believe in God,Other religion: Brahma Kumari,Other religion: Chinese Religion,Other religion: Church of All Religion,Other religion: Confucianist,Other religion: Deist,Other religion: Druid,Other religion: Druze,Other religion: Eckankar,Other religion: Heathen,Other religion: Jain,Other religion: Mixed Religion,Other religion: Mysticism,Other religion: Native American Church,Other religion: New Age,Other religion: Occult,Other religion: Own Belief System,Other religion: Pagan,Other religion: Pantheism,Other religion: Rastafarian,Other religion: Ravidassia,Other religion: Reconstructionist,Other religion: Satanism,Other religion: Scientology,Other religion: Shamanism,Other religion: Shintoism,Other religion: Spiritual,Other religion: Spiritualist,Other religion: Taoist,Other religion: Theism,Other religion: Thelemite,Other religion: Traditional African Religion,Other religion: Unification Church,Other religion: Universalist,Other religion: Vodun,Other religion: Wicca,Other religion: Witchcraft,Other religion: Zoroastrian,Other religion: Other religions,No religion: Total,No religion: No religion,No religion: Agnostic,No religion: Atheist,No religion: Free Thinker,No religion: Heavy Metal,No religion: Humanist,No religion: Jedi Knight,No religion: Realist,Religion not stated
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,10627,6053,85,284,16,694,15,38,0,0,0,0,0,0,0,0,0,0,2,0,0,5,0,0,1,0,0,12,1,5,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,1,1,0,5,2464,2422,5,4,0,0,2,31,0,978
E05013715,15544,7801,169,302,59,669,34,91,0,1,4,0,0,0,0,0,0,0,0,0,1,17,0,0,0,0,2,8,1,7,0,0,1,1,0,1,8,11,0,0,0,1,6,0,1,2,0,4,14,4912,4809,16,16,0,0,6,65,0,1507
E05013716,16587,7401,227,313,31,888,29,126,0,3,9,0,0,0,1,2,0,0,3,0,6,14,0,0,0,2,1,15,1,12,0,0,1,0,2,1,12,10,2,0,0,3,0,0,1,9,1,1,14,5990,5816,29,21,0,2,11,111,0,1582
E05013717,16097,9495,147,705,20,1317,42,69,0,0,2,2,0,0,0,0,1,0,0,0,0,5,0,0,1,0,0,8,0,10,0,1,0,0,0,0,5,10,1,0,0,3,0,0,1,2,0,1,16,2909,2864,9,4,1,1,4,26,0,1393
E05013718,14937,7453,179,205,51,761,28,92,0,1,2,0,0,1,0,0,2,0,0,0,1,5,0,0,0,0,3,21,1,13,0,0,1,0,0,1,16,9,5,0,0,0,0,0,1,2,0,2,5,4737,4652,15,13,0,1,6,50,0,1431


Reference geographies
--

In [12]:
# For filtering of source data
reference_names = ['United Kingdom', 'Great Britain', 'England and Wales',
                   'England', 'London', 'Lewisham']
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['religion']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_references.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')

  # Filter & transform to index data
  ref = filter_nomis_data(d, years, reference_names).drop(columns=['date'])
  ref = ref.rename(columns={'geography': 'Name', 'geography code': 'Code'})
  ref.to_csv(f"{output_dir}/references_{datname}.csv", index=False)
  IPython.display.display(ref.head())

'=== religion ==='

,Name,Code,Rural Urban,All categories: Religion,Christian,Buddhist,Hindu,Jewish,Muslim (Islam),Sikh,Other religion: Total,Other religion: Animism,Other religion: Baha'i,Other religion: Believe in God,Other religion: Brahma Kumari,Other religion: Chinese Religion,Other religion: Church of All Religion,Other religion: Confucianist,Other religion: Deist,Other religion: Druid,Other religion: Druze,Other religion: Eckankar,Other religion: Heathen,Other religion: Jain,Other religion: Mixed Religion,Other religion: Mysticism,Other religion: Native American Church,Other religion: New Age,Other religion: Occult,Other religion: Own Belief System,Other religion: Pagan,Other religion: Pantheism,Other religion: Rastafarian,Other religion: Ravidassia,Other religion: Reconstructionist,Other religion: Satanism,Other religion: Scientology,Other religion: Shamanism,Other religion: Shintoism,Other religion: Spiritual,Other religion: Spiritualist,Other religion: Taoist,Other religion: Theism,Other religion: Thelemite,Other religion: Traditional African Religion,Other religion: Unification Church,Other religion: Universalist,Other religion: Vodun,Other religion: Wicca,Other religion: Witchcraft,Other religion: Zoroastrian,Other religion: Other religions,No religion: Total,No religion: No religion,No religion: Agnostic,No religion: Atheist,No religion: Free Thinker,No religion: Heavy Metal,No religion: Humanist,No religion: Jedi Knight,No religion: Realist,Religion not stated
0,England,E92000001,Total,53012456,31479876,238626,806199,261282,2660116,420196,227825,487,4746,2827,434,174,380,116,1142,3946,504,367,1867,20193,21907,192,119,665,474,1842,53172,2105,7657,11045,223,1800,2361,612,1041,12991,36370,3916,791,176,584,435,862,198,11026,1193,4055,12830,13114232,12866490,30572,27761,490,5974,14252,168373,320,3804104
1,London,E12000007,Total,8173941,3957984,82026,411291,148602,1012823,126134,47970,92,1229,966,194,68,103,33,244,361,397,156,202,15020,3139,53,8,126,77,331,4943,348,2629,1575,54,232,321,88,408,2813,3242,783,187,36,252,243,161,52,1023,120,2235,3426,1694372,1658839,5678,5439,77,732,2235,21337,35,692739
2,Lewisham,E09000023,Total,275885,145588,3664,6562,643,17759,531,1478,4,25,33,3,7,3,4,5,17,5,8,12,12,121,4,2,8,7,21,259,20,172,6,2,9,7,7,17,137,153,34,5,2,25,33,6,5,62,4,25,187,75155,73582,251,219,3,14,126,959,1,24505


Derivatives
==

Religion
--

In [13]:
religion = pd.read_csv(f"{output_dir}/census11_religion_oa11_london.csv")
religion.columns

Index(['date', 'geography', 'geography code', 'Rural Urban',
       'Religion: All categories: Religion; measures: Value',
       'Religion: Christian; measures: Value',
       'Religion: Buddhist; measures: Value',
       'Religion: Hindu; measures: Value', 'Religion: Jewish; measures: Value',
       'Religion: Muslim (Islam); measures: Value',
       'Religion: Sikh; measures: Value',
       'Religion: Other religion: Total; measures: Value',
       'Religion: Other religion: Animism; measures: Value',
       'Religion: Other religion: Baha'i; measures: Value',
       'Religion: Other religion: Believe in God; measures: Value',
       'Religion: Other religion: Brahma Kumari; measures: Value',
       'Religion: Other religion: Chinese Religion; measures: Value',
       'Religion: Other religion: Church of All Religion; measures: Value',
       'Religion: Other religion: Confucianist; measures: Value',
       'Religion: Other religion: Deist; measures: Value',
       'Religion: Other 

In [14]:
# Segmentation of categories
def religion_groups(religion, fixed_columns):
  d = religion[fixed_columns].copy()
  total = religion['All categories: Religion']

  d['Christian'] = religion['Christian'] * 100. / total
  d['Buddhist'] = religion['Buddhist'] * 100. / total
  d['Hindu'] = religion['Hindu'] * 100 / total
  d['Jewish'] = religion['Jewish'] * 100 / total
  d['Muslim'] = religion['Muslim (Islam)'] * 100 / total
  d['Sikh'] = religion['Sikh'] * 100 / total

  d['Other Religion'] = religion['Other religion: Total'] * 100 / total

  d['No Religion'] = religion['No religion: Total'] * 100 / total

  d['Not Stated'] = religion['Religion not stated'] * 100 / total

  return d#.round(1)

In [15]:
# Lewisham
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  religion = pd.read_csv(f"{output_dir}/lbl_religion_{geog}.csv")
  d = religion_groups(religion, [geog_colname])
  d.to_csv(f"{output_dir}/lbl_religion_groups_{geog}.csv", index=False)
  IPython.display.display(d.head())

'=== oa11 ==='

,OA11CD,Christian,Buddhist,Hindu,Jewish,Muslim,Sikh,Other Religion,No Religion,Not Stated
0,E00016403,49.453552,3.825137,4.644809,0.0,15.300546,0.000000,0.273224,20.218579,6.284153
1,E00016437,60.780669,4.646840,0.557621,0.0,9.293680,0.557621,0.000000,16.356877,7.806691
2,E00016439,63.881402,1.347709,1.617251,0.0,8.625337,0.000000,1.617251,16.172507,6.738544
3,E00016442,58.474576,1.694915,1.694915,0.0,18.926554,0.282486,0.000000,12.429379,6.497175
4,E00016399,57.296137,3.862661,0.000000,0.0,15.879828,1.716738,0.858369,15.021459,5.364807


'=== wd22 ==='

,WD22CD_proposed,Christian,Buddhist,Hindu,Jewish,Muslim,Sikh,Other Religion,No Religion,Not Stated
0,E05013714,56.958690,0.799849,2.672438,0.150560,6.530535,0.141150,0.357580,23.186224,9.202974
1,E05013715,50.186567,1.087236,1.942872,0.379568,4.303911,0.218734,0.585435,31.600618,9.695059
2,E05013716,44.619280,1.368542,1.887020,0.186893,5.353590,0.174836,0.759631,36.112618,9.537590
3,E05013717,58.986146,0.913214,4.379698,0.124247,8.181649,0.260918,0.428651,18.071690,8.653786
4,E05013718,49.896231,1.198366,1.372431,0.341434,5.094731,0.187454,0.615920,31.713195,9.580237


Reference geographies religion
--

In [16]:
# Reference geographies
religion_ref = pd.read_csv(f"{output_dir}/references_religion.csv")
d = religion_groups(religion_ref, ['Name', 'Code'])
d.to_csv(f"{output_dir}/references_religion_groups.csv", index=False)
IPython.display.display(d.head())



,Name,Code,Christian,Buddhist,Hindu,Jewish,Muslim,Sikh,Other Religion,No Religion,Not Stated
0,England,E92000001,59.382037,0.450132,1.520773,0.492869,5.017907,0.792636,0.429757,24.738020,7.175868
1,London,E12000007,48.421979,1.003506,5.031734,1.817997,12.390877,1.543123,0.586865,20.728948,8.474969
2,Lewisham,E09000023,52.771263,1.328090,2.378527,0.233068,6.437102,0.192472,0.535730,27.241423,8.882324
